## Analysis of cross-regulation predictions of SRSF7
### p-value & peaks not exons centered

In [2]:
# updated pipeline logic should be described here

In [44]:
# packages
import pandas as pd
import seaborn as sns

In [45]:
# data
srsf7_targets = pd.read_table('./data/outputs/new_cross_peaks_nmd_SRSF7.tsv', header=None)

NMD_KD = pd.read_table('./data/input_data/upf1xrn1_deltaPSI.tsv')

for_eCLIPs = pd.read_table('./data/outputs/exon_peaks.bed', header=None)
# gcn_data
srsf7_gcn = pd.read_table('./srsf7.tsv')

In [46]:
NMD_KD.head()

,id,gene,deltaPSI.UPF1,deltaPSIc.UPF1,z.UPF1,p.UPF1,q.UPF1,deltaPSI.SMG6,deltaPSIc.SMG6,z.SMG6,p.SMG6,q.SMG6,deltaPSIc,p,q
0,chr1_10003307_10003387_-,LZIC,0.15,0.155,1.16,0.91,0.0,NaN,NaN,NaN,NaN,NaN,0.155,0.91,0.0
1,chr1_100329942_100330145_+,AGL,0.08,0.085,0.72,0.63,0.0,0.08,0.085,0.68,0.61,0.0,0.085,1.24,0.0
2,chr1_100345479_100345602_+,AGL,-0.11,-0.105,-0.48,0.50,0.0,NaN,NaN,NaN,NaN,NaN,-0.105,0.50,0.0
3,chr1_10035650_10035833_+,NMNAT1,-0.04,-0.025,-0.10,0.34,0.0,-0.02,-0.005,0.10,0.34,0.0,-0.015,0.68,0.0
4,chr1_10041314_10041824_+,NMNAT1,0.05,0.065,0.59,0.56,0.0,NaN,NaN,NaN,NaN,NaN,0.065,0.56,0.0


In [47]:
# nmd_kd parsing
nmd_kd = NMD_KD[['id', 'gene', 'deltaPSI.UPF1','p.UPF1','q.UPF1','deltaPSI.SMG6', 'p.SMG6', 'q.SMG6', 'p', 'q']]
nmd_kd = nmd_kd.rename(columns={'id':'exon', 'gene':'target', 'p':'NMD_KD_p', 'q':'NMD_KD_q'})

nmd_kd.head()

,exon,target,deltaPSI.UPF1,p.UPF1,q.UPF1,deltaPSI.SMG6,p.SMG6,q.SMG6,NMD_KD_p,NMD_KD_q
0,chr1_10003307_10003387_-,LZIC,0.15,0.91,0.0,NaN,NaN,NaN,0.91,0.0
1,chr1_100329942_100330145_+,AGL,0.08,0.63,0.0,0.08,0.61,0.0,1.24,0.0
2,chr1_100345479_100345602_+,AGL,-0.11,0.50,0.0,NaN,NaN,NaN,0.50,0.0
3,chr1_10035650_10035833_+,NMNAT1,-0.04,0.34,0.0,-0.02,0.34,0.0,0.68,0.0
4,chr1_10041314_10041824_+,NMNAT1,0.05,0.56,0.0,NaN,NaN,NaN,0.56,0.0


In [48]:
# eCLIPs parsing
eCLIPs = for_eCLIPs.iloc[:, [3,6,10,12,13,14,15,16]]
eCLIPs.columns = ['exon', 'target', 'factor', 'strand', 'eCLIPs cell line', 'rep', 'eCLIP_p-value', 'eCLIP_logFC']
eCLIPs.head()

,exon,target,factor,strand,eCLIPs cell line,rep,eCLIP_p-value,eCLIP_logFC
0,chr1_137621_139379_-,AL627309.1,ILF3,-,K562,rep02,2.421904,5.174009
1,chr1_137621_139379_-,AL627309.1,HNRNPK,-,K562,rep02,3.064845,5.305207
2,chr1_137621_139379_-,AL627309.1,BUD13,-,K562,rep02,2.653889,3.067285
3,chr1_137621_139379_-,AL627309.1,HNRNPK,-,K562,rep01,1.620957,1.052859
4,chr1_137621_139379_-,AL627309.1,PUM1,-,K562,rep02,3.858528,5.736128


In [49]:
srsf7_targets = srsf7_targets.iloc[:, [0,1,2,3,5,9,12]]
srsf7_targets.columns = ['chrom', 'peak_start', 'peak_end', 'factor', 'strand', 'exon', \
                         'target']

srsf7_targets.head()

,chrom,peak_start,peak_end,factor,strand,exon,target
0,chr1,1203271,1203356,SRSF7,-,chr1_1203113_1203372_-,UBE2J2
1,chr1,1203382,1203460,SRSF7,-,chr1_1203113_1203372_-,UBE2J2
2,chr1,1203271,1203356,SRSF7,-,chr1_1203113_1203372_-,UBE2J2
3,chr1,1203382,1203460,SRSF7,-,chr1_1203113_1203372_-,UBE2J2
4,chr1,1203271,1203356,SRSF7,-,chr1_1203113_1203372_-,UBE2J2


In [50]:
# NMD KD data adding
SRSF7_added_NMD_KD = pd.merge(nmd_kd, srsf7_targets, on=['exon', 'target'])
SRSF7_added_NMD_KD.head()
SRSF7_added_NMD_KD = SRSF7_added_NMD_KD[['exon','target','factor','peak_start','peak_end',\
                                         'deltaPSI.UPF1','p.UPF1', 'q.UPF1', \
                                         'deltaPSI.SMG6', 'p.SMG6', 'q.SMG6', \
                                         'NMD_KD_p', 'NMD_KD_q']]
SRSF7_added_NMD_KD.head()

,exon,target,factor,peak_start,peak_end,deltaPSI.UPF1,p.UPF1,q.UPF1,deltaPSI.SMG6,p.SMG6,q.SMG6,NMD_KD_p,NMD_KD_q
0,chr1_10363221_10366815_+,KIF1B,SRSF7,10364632,10364674,0.28,1.61,0.08,0.23,1.35,0.0,2.96,0.08
1,chr1_10363221_10368655_+,KIF1B,SRSF7,10364632,10364674,0.28,1.61,0.08,0.23,1.35,0.0,2.96,0.08
2,chr1_113247722_113248874_-,RHOC,SRSF7,113249000,113249053,0.13,0.82,0.00,0.04,0.47,0.0,1.29,0.00
3,chr1_113247722_113248874_-,RHOC,SRSF7,113249024,113249107,0.13,0.82,0.00,0.04,0.47,0.0,1.29,0.00
4,chr1_1203113_1203372_-,UBE2J2,SRSF7,1203271,1203356,-0.17,0.67,0.00,-0.26,1.22,0.0,1.89,0.00


In [53]:
# then we add eCLIPs data
SRSF7_added_NMD_KD_added_eCLIPs = pd.merge(SRSF7_added_NMD_KD, eCLIPs, 
                                           on=['exon', 'target', 'factor'])
SRSF7_added_NMD_KD_added_eCLIPs.shape

(11479, 18)

In [54]:
SRSF7_added_NMD_KD_added_eCLIPs.head()

,exon,target,factor,peak_start,peak_end,deltaPSI.UPF1,p.UPF1,q.UPF1,deltaPSI.SMG6,p.SMG6,q.SMG6,NMD_KD_p,NMD_KD_q,strand,eCLIPs cell line,rep,eCLIP_p-value,eCLIP_logFC
0,chr1_10363221_10366815_+,KIF1B,SRSF7,10364632,10364674,0.28,1.61,0.08,0.23,1.35,0.0,2.96,0.08,+,HepG2,rep02,2.930191,1.309910
1,chr1_10363221_10368655_+,KIF1B,SRSF7,10364632,10364674,0.28,1.61,0.08,0.23,1.35,0.0,2.96,0.08,+,HepG2,rep02,2.930191,1.309910
2,chr1_113247722_113248874_-,RHOC,SRSF7,113249000,113249053,0.13,0.82,0.00,0.04,0.47,0.0,1.29,0.00,-,HepG2,rep01,3.023052,1.694264
3,chr1_113247722_113248874_-,RHOC,SRSF7,113249000,113249053,0.13,0.82,0.00,0.04,0.47,0.0,1.29,0.00,-,K562,rep01,2.672946,1.222546
4,chr1_113247722_113248874_-,RHOC,SRSF7,113249024,113249107,0.13,0.82,0.00,0.04,0.47,0.0,1.29,0.00,-,HepG2,rep01,3.023052,1.694264


In [79]:
# merging with gcn data
SRSF7_all = pd.merge(SRSF7_added_NMD_KD_added_eCLIPs, srsf7_gcn, how='left', on=['factor','target'])
SRSF7_all = SRSF7_all.rename(columns={'value':'gcn_value'})
filtered_by_gcn = filtered_by_gcn[filtered_by_gcn['gcn_value'].notna() == True]
targets = set(filtered_by_gcn.target)
targets


{'CLK1',
 'HNRNPA2B1',
 'HNRNPD',
 'HNRNPH1',
 'RBM39',
 'RBM5',
 'SREK1',
 'SRSF1',
 'SRSF10',
 'SRSF11',
 'SRSF4',
 'SRSF6',
 'TIA1',
 'TRA2A',
 'U2AF1'}

In [87]:
#here to make one exon with average values
filtered_table = filtered_by_gcn.groupby(['exon', 'peak_start', 'peak_end', 'target']).mean().round(2)
all_table = SRSF7_all.groupby(['exon', 'peak_start', 'peak_end', 'target']).mean().round(2)

deltaPSI.UPF1  \
exon                       peak_start peak_end  target                     
chr17_56082775_56082961_-  56082270   56082360  SRSF1              -0.26   
                           56082301   56082317  SRSF1              -0.26   
                           56082755   56082797  SRSF1              -0.26   
                           56082797   56082928  SRSF1              -0.26   
                           56082808   56082904  SRSF1              -0.26   
                           56083160   56083260  SRSF1              -0.26   
                           56083165   56083264  SRSF1              -0.26   
                           56083186   56083248  SRSF1              -0.26   
chr1_24295573_24297929_-   24298334   24298441  SRSF10             -0.09   
chr1_24296854_24297929_-   24298334   24298441  SRSF10             -0.09   
chr1_24304401_24304763_-   24304412   24304486  SRSF10              0.59   
                           24304418   24304505  SRSF10              0.59   
                           24304621   24304689  SRSF10              0.59   
                           24305206   24305246  SRSF10              0.59   
chr1_29493241_29493424_-   29493267   29493317  SRSF4               0.11   
                           29493399   29493426  SRSF4               0.11   
chr1_70696778_70697253_+   70697564   70697643  SRSF11              0.16   
                           70697580   70697650  SRSF11              0.16   
chr1_70697542_70697658_+   70697564   70697643  SRSF11              0.48   
                           70697580   70697650  SRSF11              0.48   
chr20_34327315_34327469_-  34326884   34326956  RBM39               0.13   
                           34326885   34326965  RBM39               0.13   
                           34326895   34326928  RBM39               0.13   
                           34327321   34327458  RBM39               0.13   
                           34327323   34327460  RBM39               0.13   
chr20_34328447_34328519_-  34328378   34328429  RBM39               0.46   
                           34328429   34328519  RBM39               0.46   
                           34328437   34328519  RBM39               0.46   
                           34328731   34328791  RBM39               0.46   
                           34328736   34328804  RBM39               0.46   
                           34328737   34328779  RBM39               0.46   
                           34328742   34328770  RBM39               0.46   
chr20_42087793_42088060_+  42087890   42087931  SRSF6               0.43   
                           42087897   42087929  SRSF6               0.43   
                           42088409   42088465  SRSF6               0.43   
chr21_44521476_44521542_-  44521467   44521529  U2AF1              -0.05   
chr2_201724848_201724938_- 201724384  201724459 CLK1               -0.30   
                           201724907  201724973 CLK1               -0.30   
chr2_70452460_70452525_-   70452469   70452535  TIA1                0.07   
chr2_70455476_70455594_-   70455508   70455561  TIA1                0.06   
chr3_50147812_50147896_+   50147805   50147830  RBM5               -0.11   
chr4_83277690_83277836_-   83277943   83278014  HNRNPD             -0.12   
chr5_179042547_179042596_- 179042566  179042595 HNRNPH1             0.02   
chr5_65454637_65454760_+   65454674   65454766  SREK1               0.21   
                           65455066   65455151  SREK1               0.21   
chr7_23561751_23562051_-   23561326   23561372  TRA2A               0.62   
                           23561342   23561414  TRA2A               0.62   
                           23561733   23561790  TRA2A               0.62   
                           23561789   23561875  TRA2A               0.62   
                           23561790   23561875  TRA2A               0.62   
                           23561814   23561868  TRA2A               0.62   
chr7_23561973_23562051_-   2356

In [88]:
# saving output as tsv
filtered_table.to_csv('data/outputs/SRSF7_filtered_new.tsv', sep = '\t')
all_table.to_csv('data/outputs/SRSF7_all_new.tsv', sep = '\t')
